# Project 3 Part 1 (Core)

Rodrigo Arguello-Serrano

# Business Problem

>For this project, I produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. I will use this database to analyze what makes a movie successful and provide recommendations to the stakeholder on how to make a successful movie.

# Specifications


> - Exclude any movie with missing values for genre or runtime
> - Include only full-length movies (titleType = "movie").
> - Include only fictional movies (not from documentary genre)
> - Include only movies that were released 2000 - 2021 (include 2000 and 2021)
> - Include only movies that were released in the United States

<a name="1.1"></a>
<h2> 1. 📒Data Dictionary: IMDb Dataset Details </h2>



**title.akas.tsv.gz** - Contains the following information for titles:
titleId (string) - a tconst, an alphanumeric unique identifier of the title<br>
ordering (integer) – a number to uniquely identify rows for a given titleId<br>
title (string) – the localized title<br>
region (string) - the region for this version of the title<br>
language (string) - the language of the title<br>
types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay".<br>
attributes (array) - Additional terms to describe this alternative title, not enumerated<br>
isOriginalTitle (boolean) – 0: not original title; 1: original title
<br>
<br>
**title.basics.tsv.gz** - Contains the following information for titles:<br>
tconst (string) - alphanumeric unique identifier of the title<br>
titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)<br>
primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release<br>
originalTitle (string) - original title, in the original language<br>
isAdult (boolean) - 0: non-adult title; 1: adult title<br>
startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year<br>
endYear (YYYY) – TV Series end year. ‘\N’ for all other title types<br>
runtimeMinutes – primary runtime of the title, in minutes<br>
genres (string array) – includes up to three genres associated with the title
<br><br>
**title.ratings.tsv.gz** – Contains the IMDb rating and votes information for titles<br>
tconst (string) - alphanumeric unique identifier of the title<br>
averageRating – weighted average of all the individual user ratings<br>
numVotes - number of votes the title has received<br>
[link text](http://url)
This is [My Kaggle Account](https://www.kaggle.com/ahmetekiz)

## 2. Import Modules and Using the TMDB API

In [2]:
#Import Modules
import pandas as pd
import numpy as np
import os, time, json

import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook

FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.DS_Store',
 'title_akas.csv',
 'title_basics.csv.gz',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

### 2.1 Using Your API Credentials

In [34]:
import json
with open('/Users/rodrigoarguelloserrano/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()


dict_keys(['api-key'])

### 2.2 Set the API_KEY

In [35]:
#import the package and set the API_KEY
tmdb.API_KEY =  login['api-key']

### 2.3 proof-of-concept: List of the Years to Extract from the API and Error List

In [36]:
YEARS_TO_GET = [2000,2001]

In [37]:
errors = []

## 3. Upload datasets

In [3]:
# Assigning dataset urls
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [4]:
# Reading data and assigning to dataframe: basics
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
# Repalcing missing values key \N with numpy nan objects
basics.replace({'\\N':np.nan},inplace=True)

In [5]:
# Reading data and assigning to dataframe: akas
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
# Repalcing missing values key \N with numpy nan objects
akas.replace({'\\N':np.nan},inplace=True)

In [6]:
# Reading data and assigning to dataframe: 
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
# Repalcing missing values key \N with numpy nan objects
ratings.replace({'\\N':np.nan},inplace=True)

## 4. Akas data clean up

In [7]:
# keep only US movies.
akas = akas[akas['region']=='US']

In [8]:
akas.shape

(1418920, 8)

## 5. Basics data clean up

In [9]:
#Eliminate movies that are null for runtimeMinutes and generes
basics=basics.dropna(subset=['runtimeMinutes','genres'])

In [10]:
basics.shape

(2758861, 9)

In [11]:
#keep only titleType==Movie
basics=basics[basics['titleType']=='movie']

In [12]:
basics.shape

(378331, 9)

In [13]:
basics['startYear'] = basics['startYear'].astype(float)

In [14]:
#keep startYear 2000-2022
basics = basics[(basics['startYear']<=2022.0)&(basics['startYear']>=2000.0)]

In [15]:
basics.shape

(221980, 9)

In [16]:
# Eliminate movies that include "Documentary" in genre (see tip below)
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics= basics[~is_documentary]

In [17]:
basics['tconst'].isin(akas['titleId']).value_counts()

True     85744
False    60995
Name: tconst, dtype: int64

In [18]:
#Keep only US movies
keepers =basics['tconst'].isin(akas['titleId'])
basics = basics[keepers]

## 5. Ratings data clean up

In [19]:
#Keep only US movies
keepers_r =ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers_r]

In [20]:
# Dropping all rows with nan values from akka
akas = akas.dropna()

## 7. Checking Dataframe Info

In [21]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85744 entries, 34803 to 9663938
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          85744 non-null  object 
 1   titleType       85744 non-null  object 
 2   primaryTitle    85744 non-null  object 
 3   originalTitle   85744 non-null  object 
 4   isAdult         85744 non-null  object 
 5   startYear       85744 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  85744 non-null  object 
 8   genres          85744 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.5+ MB


In [22]:
# Dropping all rows with nan values from akka
akas = akas.dropna()

In [23]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 102334 to 1051596
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleId          22 non-null     object
 1   ordering         22 non-null     int64 
 2   title            22 non-null     object
 3   region           22 non-null     object
 4   language         22 non-null     object
 5   types            22 non-null     object
 6   attributes       22 non-null     object
 7   isOriginalTitle  22 non-null     object
dtypes: int64(1), object(7)
memory usage: 1.5+ KB


In [24]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490971 entries, 0 to 1285659
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         490971 non-null  object 
 1   averageRating  490971 non-null  float64
 2   numVotes       490971 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.0+ MB


## 8. Saving Dataframes

### 8.1 Basics Dataframe

In [25]:
## Save current dataframe to file.
basics.to_csv(f"{FOLDER}title_basics.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
basics = pd.read_csv(f"{FOLDER}title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


### 8.2 Akas Dataframe

In [27]:
## Save current dataframe to file.
akas.to_csv(f"{FOLDER}title_akas.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
akas = pd.read_csv(f"{FOLDER}title_akas.csv.gz",compression='gzip')
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0022542,1,Di shtime fun Yisroel,US,yi,alternative,YIVO translation,0
1,tt0024265,4,Geleb un gelakht,US,yi,alternative,modern translation,0
2,tt0024751,9,Avram Ovenu,US,yi,alternative,YIVO translation,0
3,tt0026010,3,Der yidishe Kenigen Lir,US,yi,alternative,YIVO translation,0
4,tt0027911,1,Libe un Laydnshaft,US,yi,alternative,modern translation,0


### 8.3 Ratings Dataframe

In [28]:
## Save current dataframe to file.
ratings.to_csv(f"{FOLDER}title_ratings.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
ratings = pd.read_csv(f"{FOLDER}title_ratings.csv.gz",compression='gzip')
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1957
1,tt0000002,5.8,263
2,tt0000005,6.2,2594
3,tt0000006,5.1,177
4,tt0000007,5.4,812


## 9. Helper Functiona

In [29]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [31]:
def get_movie_with_rating(movie_id):
# Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            info['certification'] = c['certification']
    return info

## 10. Loop Funtion to retrive YEARS with ratings

In [38]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1434 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1551 [00:00<?, ?it/s]

- Total errors: 455


> Testing our function by looping through a list of ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook")

In [39]:
test_ids = ["tt0848228", "tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28263,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,10186,PG-13


## 11. Exploratory Data Analysis

### 11.1 Contatening Data from YEARS 2000 and 2001

In [42]:
data_2000 = pd.read_csv(f"{FOLDER}final_tmdb_data_2000.csv.gz")

In [43]:
data_2001 = pd.read_csv(f"{FOLDER}final_tmdb_data_2001.csv.gz")

In [45]:
type(data_2000)

pandas.core.frame.DataFrame

In [46]:
tmdb_results_combined = pd.concat([data_2000,data_2001])

### 11.2 Save a final merged .csv.gz of all of the tmdb api data: ```tmdb_results_combined.csv.gz```

In [47]:
tmdb_results_combined.to_csv(f"{FOLDER}tmdb_results_combined.csv.gz", compression="gzip", index=False)

### 11.3 How many movies had at least some valid financial information (values > 0 for budget OR revenue)?

### 11.4 How many movies are there in each of the certification categories (G/PG/PG-13/R)?

### 11.5 What is the average revenue per certification category?

### 11.6 What is the average budget per certification category?